In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%autoreload 2

In [3]:
%matplotlib inline
import sys
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator

LANDSCAPE_TYPES ={"RNA": [3,2], 'TF':[1]}
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

Using TensorFlow backend.


B1L14RNA1 loaded



../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## DQN Explorer

In [7]:
from explorers.dqn_explorer import DQN_Explorer 

num_of_rounds = 5

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(seq_len=14)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.14554841726711917
Q_Network(
  (linear1): Linear(in_features=112, out_features=56, bias=True)
  (bn1): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=56, out_features=14, bias=True)
  (bn2): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=14, out_features=1, bias=True)
)
round: 1, cost: 83, evals: 82, top: 0.6947876817534437
round: 2, cost: 163, evals: 162, top: 0.6947876817534437
round: 3, cost: 232, evals: 231, top: 0.6947876817534437
round: 4, cost: 298, evals: 297, top: 0.6947876817534437


In [ ]:
evaluator_dqn=Evaluator(dqn_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=1)

loading landscapes RNA: [3, 2], TF:[1]
B1L14RNA2 loaded
B1L100RNA1 loaded


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


1 TF landscapes loaded.
loading complete
Running on B1L14RNA2
start seq startRNAL14_0
Q_Network(
  (linear1): Linear(in_features=112, out_features=56, bias=True)
  (bn1): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=56, out_features=14, bias=True)
  (bn2): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=14, out_features=1, bias=True)
)
Q_Network(
  (linear1): Linear(in_features=112, out_features=56, bias=True)
  (bn1): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=56, out_features=14, bias=True)
  (bn2): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=14, out_features=1, bias=True)
)
Q_Network(
  (linear1): Linear(in_features=112, out_features=56, bias=True)
  (bn1): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_run

## BO Explorer

In [8]:
%autoreload 2
from explorers.bo_explorer import BO_Explorer

num_of_rounds = 5
arch1 = NLNN(len(wt), hidden_dims=10, epochs=20)
arch2 = NLNN(len(wt), hidden_dims=20, epochs=20)
arch3 = NLNN(len(wt), hidden_dims=30, epochs=20)
arch4 = NLNN(len(wt), hidden_dims=40, epochs=20)
# ensemble we will use for BO Explorer
nn_model_1 = NN_model(landscape_oracle, arch1)
nn_model_2 = NN_model(landscape_oracle, arch2)
nn_model_3 = NN_model(landscape_oracle, arch3)
nn_model_4 = NN_model(landscape_oracle, arch4)
models = [nn_model_1, nn_model_2, nn_model_3, nn_model_4]
ensemble_model = Ensemble_models(models)
ensemble_model.update_model(training_sequences)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [9]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer(seq_len=14)
bo_explorer.debug = True
bo_explorer.set_model(ensemble_model)
bo_explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 1, evals: 0, top: 0.14554841726711917
round: 1, cost: 61, evals: 2904, top: 0.2389190862061472
round: 2, cost: 132, evals: 6124, top: 0.55747785908442
round: 3, cost: 197, evals: 9240, top: 0.7112648709496348
round: 4, cost: 296, evals: 13044, top: 0.7112648709496348


In [ ]:
evaluator_bo=Evaluator(bo_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=1)